In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset = pd.read_csv('data/Loan_Application.csv')
X = dataset.iloc[:, 1:11].values
y = dataset.iloc[:, 11].values

In [3]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

In [4]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### ANN


In [6]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [7]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))

C:\tools\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform")`
  """


In [8]:
# Adding regularization
#classifier.add(Dropout(p = 0.1))   

Dropout parameter of 0.1 means 10% of the neurons would be disabled at each iteration.
If required, increase by 0.1 to improve the accuracy until 0.5.
So, that means p = 1, means no neurons and that is under-fitting.
In general, don't go over 0.5.

In [9]:
# Adding the second hidden layer
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))

C:\tools\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  


In [10]:
# Adding regularization
# classifier.add(Dropout(p = 0.1))

In [11]:
# Adding the output layer
classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

C:\tools\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [12]:
# Fitting the ANN to the Training set
#classifier.fit(X_train, y_train, batch_size = 100, nb_epoch = 100)
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 5)

C:\tools\Anaconda2\envs\py36\lib\site-packages\keras\models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/5
8000/8000 [==============================] - 0s - loss: 0.4845 - acc: 0.7960     
Epoch 2/5
8000/8000 [==============================] - 0s - loss: 0.4290 - acc: 0.7960     
Epoch 3/5
8000/8000 [==============================] - 0s - loss: 0.4243 - acc: 0.8007     
Epoch 4/5
8000/8000 [==============================] - 0s - loss: 0.4198 - acc: 0.8194     
Epoch 5/5
8000/8000 [==============================] - 1s - loss: 0.4156 - acc: 0.8275     


In [15]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [16]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1568,   27],
       [ 303,  102]], dtype=int64)

In [17]:
accuracy = (cm[0][0] + cm[1][1]) / (cm[0][0] + cm[1][0] + cm[1][1] + cm[0][1])
accuracy

0.83499999999999996

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

def build_classifier(optimizer):
    classifier = Sequential()    
 
    classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu'))
    #classifier.add(Dropout(p = 0.1))
    classifier.add(Dense(output_dim = 1, init = 'uniform', activation = 'sigmoid'))
    #classifier.add(Dropout(p = 0.1))    
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) 
    return classifier
    
classifier = KerasClassifier(build_fn = build_classifier)

parameters = {'batch_size': [100, 300],
              #'epochs': [100, 500],
              'nb_epoch': [3, 5],
              'optimizer': ['adam', 'rmsprop']}

grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 2) #10 

In [ ]:
grid_search = grid_search.fit(X_train, y_train)

In [ ]:
best_parameters = grid_search.best_params_   # batch_size = 25, nb_epoch=500, optimizer = rmsprop 
best_accuracy = grid_search.best_score_    # 85.06